In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [2]:
# Load the data from a CSV file
data = pd.read_csv("chitradurga.csv")

# Remove the "time" column
data = data.drop(["time"], axis=1)

# Split the data into current and target variables
current_vars = data[["temperature_2m", "relativehumidity_2m"]]
target_vars = data[["rain", "soil_temperature_0_to_7cm", "soil_moisture"]]

# Normalize the data
current_vars = (current_vars - current_vars.mean()) / current_vars.std()
target_vars = (target_vars - target_vars.mean()) / target_vars.std()

# Split the data into training and testing sets
train_size = int(len(data) * 0.8)
train_current_vars = current_vars[:train_size]
train_target_vars = target_vars[:train_size]
test_current_vars = current_vars[train_size:]
test_target_vars = target_vars[train_size:]


In [3]:
#reshape the data
train_current_vars = np.array(train_current_vars).reshape(-1, 1, 2)
train_target_vars = np.array(train_target_vars).reshape(-1, 1, 3)
test_current_vars = np.array(test_current_vars).reshape(-1, 1, 2)
test_target_vars = np.array(test_target_vars).reshape(-1, 1, 3)

# Define the input shape
input_shape = (None, 2)

# Define the number of LSTM cells and output nodes
num_cells = 64
num_outputs = 3

# Build the LSTM model
model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(num_cells, input_shape=input_shape, activation="sigmoid"),
    tf.keras.layers.Dense(num_outputs, activation="linear")
])

# Compile the model
model.compile(loss="mean_squared_error", optimizer="Adam")


In [5]:
#set model parameters
batch_size = 32
num_epochs = 100


#train the model
history = model.fit(train_current_vars, train_target_vars, epochs=100, validation_data=(test_current_vars, test_target_vars))



Epoch 1/100
5095/5095 [==============================] - 21s 4ms/step - loss: 0.9938 - val_loss: 1.0122
Epoch 2/100
5095/5095 [==============================] - 20s 4ms/step - loss: 0.9938 - val_loss: 1.0135
Epoch 3/100
5095/5095 [==============================] - 19s 4ms/step - loss: 0.9936 - val_loss: 1.0143
Epoch 4/100
5095/5095 [==============================] - 17s 3ms/step - loss: 0.9935 - val_loss: 1.0203
Epoch 5/100
5095/5095 [==============================] - 17s 3ms/step - loss: 0.9935 - val_loss: 1.0199
Epoch 6/100
5095/5095 [==============================] - 17s 3ms/step - loss: 0.9937 - val_loss: 1.0103
Epoch 7/100
5095/5095 [==============================] - 17s 3ms/step - loss: 0.9933 - val_loss: 1.0128
Epoch 8/100
5095/5095 [==============================] - 17s 3ms/step - loss: 0.9935 - val_loss: 1.0072
Epoch 9/100
5095/5095 [==============================] - 17s 3ms/step - loss: 0.9934 - val_loss: 1.0161
Epoch 10/100
5095/5095 [==============================] - 16s 3m

In [6]:
#save the model
model.save("chitradurga1.h5")


In [7]:
#LOAD THE MODEL
model = tf.keras.models.load_model("chitradurga1.h5")


In [8]:
#make some test predictions
predictions = model.predict(test_current_vars)



1274/1274 [==============================] - 1s 1ms/step


In [9]:
#print the predictions
print(predictions)


[[-0.00888728  0.00304751 -0.02481037]
 [-0.00763468 -0.00014901 -0.02144819]
 [-0.00775682 -0.00181108 -0.02142365]
 ...
 [-0.01658102  0.03694058 -0.05686431]
 [-0.01688932  0.02955078 -0.0575743 ]
 [-0.01713032  0.02065526 -0.05699842]]


In [21]:
#plot the predictions of rain,soil_temperature_0_to_7cm,soil_moisture
plt.plot(predictions[:, :, 0], label="Rain")
plt.plot(predictions[:, :, 1], label="Soil Temperature")
plt.plot(predictions[:, :, 2], label="Soil Moisture")
plt.legend()

#check model accuracy for all the variables in the dataset
print(model.evaluate(test_current_vars, test_target_vars))



IndexError: too many indices for array: array is 2-dimensional, but 3 were indexed

In [15]:
#test for single input
test_input = np.array([[0.5, 0.5]])
test_input = test_input.reshape((1, 1, 2))
print(model.predict(test_input))

#check model accuracy using shaped data
#model.evaluate(test_current_vars, test_target_vars)

#print the model accuracy calculated for 100 percent
print(model.evaluate(test_current_vars, test_target_vars))




1/1 [==============================] - 0s 12ms/step
[[ 0.00298148  0.0291717  -0.00465049]]
1274/1274 [==============================] - 2s 1ms/step - loss: 1.0128
1.0128110647201538


ValueError: Found array with dim 3. None expected <= 2.